In [15]:
#Some nice imports
import requests
import json
import numpy as np
import pandas as pd
from pandas import json_normalize
import geopandas as gpd


In [16]:
df = pd.read_csv('coordinate_df.csv')

### Add `properties.group_name` column

In [17]:
df.rename(columns={"cleaned_type":"properties.group_name"}, inplace=True)

In [18]:
df['properties.group_name'] = df['properties.group_name'].astype('category')

### Add `properties.radius` column

In [19]:
# df.rename(columns={"capacity_MW":"properties.radius"}, inplace=True)
# df['properties.radius'] = df['properties.radius'].round(2)

df['properties.radius'] = 5 * np.log10(df.capacity_MW)

In [20]:
df

plant_count                     geometry.coordinates  capacity_MW  \
0              6                [129.0464035, 37.0845505]        121.2   
1              5                [128.7209909, 37.7031357]        112.0   
2              5                [128.4932052, 37.1656066]         49.5   
3              5                [126.7063341, 33.3988071]         94.9   
4              4  [129.51648868036636, 35.53865916003904]       1932.0   
..           ...                                      ...          ...   
951            1                [129.4338864, 36.5463932]          9.9   
952            1                [129.3237859, 36.4590165]         18.0   
953            1                [129.3004787, 36.4714752]         45.0   
954            1                [129.4074059, 36.4667267]         56.1   
955            1                [129.2754707, 36.0854411]         20.0   

    properties.group_name                                 properties.article  \
0                     gas  <div class='plant_list'><P>Type: gas\nCapacity...   
1                    wind  <div class='plant_list'><P>Type: wind\nCapacit...   
2                     gas  <div class='plant_list'><P>Type: gas\nCapacity...   
3                   solar  <div class='plant_list'><P>Type: solar\nCapaci...   
4                    wind  <div class='plant_list'><P>Type: wind\nCapacit...   
..                    ...                                                ...   
951             bioenergy  <div class='plant_list'><P>Type: bioenergy\nCa...   
952                  wind  <div class='plant_list'><P>Type: wind\nCapacit...   
953                  wind  <div class='plant_list'><P>Type: wind\nCapacit...   
954                  wind  <div class='plant_list'><P>Type: wind\nCapacit...   
955                  wind  <div class='plant_list'><P>Type: wind\nCapacit...   

     properties.radius  
0            10.417513  
1            10.246090  
2             8.473026  
3             9.886331  
4            16.430036  
..                 ...  
951           4.978176  
952           6.276363  
953           8.266063  
954           8.744814  
955           6.505150  

[956 rows x 6 columns]

### Add `properties.name` column

In [21]:
df.rename(columns={"company":"properties.name"}, inplace=True)

### Add `properties.group_id` column

In [22]:
df['properties.group_name'].cat.categories

Index(['bioenergy', 'coal', 'gas', 'hydro', 'nuclear', 'others', 'solar',
       'wind'],
      dtype='object')

In [23]:
df['properties.group_id'] = df['properties.group_name'].replace(
    {
        cat: n + 1
        for n, cat
        in enumerate(df['properties.group_name'].cat.categories)
    }
)

### Change column names to include `properties.`

In [24]:
df.rename(columns={"plant_count":"properties.COUNT"}, inplace=True)

### Add `properties.headline` column

In [25]:
def add_text(plant_count, group_name, capacity):
    return str(plant_count) + " " + group_name + " plant(s). Capacity: " + str(capacity) + " MW" 
                                                          

In [26]:
df['properties.headline'] = df.apply(lambda x: add_text(x['properties.COUNT'], x['properties.group_name'], x['capacity_MW']), axis=1)



In [27]:
df.sort_values(by=['properties.radius'], ascending=False).head(30)

properties.COUNT                     geometry.coordinates  capacity_MW  \
364                 1                [126.5191246, 36.4007416]       5800.0   
94                  2                [126.5096353, 37.0326652]       5000.0   
379                 1                [126.2247896, 36.8948887]       4000.0   
71                  2                [126.5908916, 36.9819022]       3150.0   
742                 1                [128.4245023, 36.7573156]       2800.0   
910                 1                [129.3845078, 37.0866593]       2800.0   
909                 1                [129.3422373, 36.8228854]       2800.0   
164                 1                [129.3584669, 35.5154368]       2800.0   
612                 1                [128.9541928, 37.7280308]       2120.0   
534                 1                [129.1770184, 37.4147184]       2100.0   
888                 1                [128.1164472, 34.9149767]       2100.0   
378                 1                [126.2312041, 36.8979989]       2100.0   
488                 1  [126.48227263966949, 35.81943934257905]       2100.0   
624                 1                [129.3432037, 37.2325164]       2000.0   
783                 1                [129.4430326, 35.7910787]       2000.0   
365                 1                [126.5692554, 36.3908376]       2000.0   
800                 1                [129.2991326, 35.3323776]       2000.0   
4                   4  [129.51648868036636, 35.53865916003904]       1932.0   
52                  2                [128.4068799, 34.8996122]       1840.0   
513                 1                [126.7954201, 37.8214984]       1800.0   
253                 1                [126.2949081, 37.7984052]       1740.0   
41                  2                [127.1891285, 37.9915306]       1545.0   
509                 1                [127.1009159, 37.9067614]       1500.0   
24                  3  [129.35849908473662, 35.51559397107106]       1387.7   
566                 1                [127.0927885, 37.1764742]       1283.9   
213                 1  [126.60847324245631, 37.53605097894146]       1147.0   
576                 1                [128.1126122, 34.9131431]       1120.0   
15                  3                [129.3146114, 35.4346298]       1058.0   
303                 1                [127.0089277, 36.7317626]       1000.0   
373                 1                [126.4973423, 36.1409666]       1000.0   

    properties.group_name                                 properties.article  \
364                  coal  <div class='plant_list'><P>Type: coal\nCapacit...   
94                   coal  <div class='plant_list'><P>Type: coal\nCapacit...   
379                  coal  <div class='plant_list'><P>Type: coal\nCapacit...   
71                   coal  <div class='plant_list'><P>Type: coal\nCapacit...   
742               nuclear  <div class='plant_list'><P>Type: nuclear\nCapa...   
910               nuclear  <div class='plant_list'><P>Type: nuclear\nCapa...   
909               nuclear  <div class='plant_list'><P>Type: nuclear\nCapa...   
164               nuclear  <div class='plant_list'><P>Type: nuclear\nCapa...   
612                  coal  <div class='plant_list'><P>Type: coal\nCapacit...   
534                  coal  <div class='plant_list'><P>Type: coal\nCapacit...   
888                  coal  <div class='plant_list'><P>Type: coal\nCapacit...   
378                  coal  <div class='plant_list'><P>Type: coal\nCapacit...   
488                 solar  <div class='plant_list'><P>Type: solar\nCapaci...   
624                  coal  <div class='plant_list'><P>Type: coal\nCapacit...   
783               nuclear  <div class='plant_list'><P>Type: nuclear\nCapa...   
365                  coal  <div class='plant_list'><P>Type: coal\nCapacit...   
800               nuclear  <div class='plant_list'><P>Type: nuclear\nCapa...   
4                    wind  <div class='plant_list'><P>Type: wind\nCapacit...   
52                    gas  <div cla


**color:** This will set the color for every shape. Here we are making semi-random colors for every single shape...Not a good thing to do. But definitely have a lot of funWith your color algorithms here. You want the colors to reflect different ranges of values. Random is the last thing you want to do. But the function below builds random hexadecimal color values.


In [28]:
df[df['properties.group_id']==7]

properties.COUNT                     geometry.coordinates  capacity_MW  \
3                   5                [126.7063341, 33.3988071]         94.9   
5                   4                [126.7356547, 33.4806289]         63.1   
11                  3                 [126.559637, 35.9599248]        270.0   
14                  3                 [126.669278, 37.4380623]         20.1   
20                  3                [126.7464887, 35.0046287]         35.0   
..                ...                                      ...          ...   
898                 1                [128.8364784, 36.8267892]         55.0   
916                 1   [128.66822605591983, 36.3656026892897]         40.0   
918                 1                [128.7614838, 36.4334642]         50.0   
935                 1                [128.7449683, 36.5870676]          9.9   
936                 1  [128.83163024243228, 36.51575239781046]          9.8   

    properties.group_name                                 properties.article  \
3                   solar  <div class='plant_list'><P>Type: solar\nCapaci...   
5                   solar  <div class='plant_list'><P>Type: solar\nCapaci...   
11                  solar  <div class='plant_list'><P>Type: solar\nCapaci...   
14                  solar  <div class='plant_list'><P>Type: solar\nCapaci...   
20                  solar  <div class='plant_list'><P>Type: solar\nCapaci...   
..                    ...                                                ...   
898                 solar  <div class='plant_list'><P>Type: solar\nCapaci...   
916                 solar  <div class='plant_list'><P>Type: solar\nCapaci...   
918                 solar  <div class='plant_list'><P>Type: solar\nCapaci...   
935                 solar  <div class='plant_list'><P>Type: solar\nCapaci...   
936                 solar  <div class='plant_list'><P>Type: solar\nCapaci...   

     properties.radius properties.group_id  \
3             9.886331                   7   
5             9.000147                   7   
11           12.156819                   7   
14            6.515980                   7   
20            7.720340                   7   
..                 ...                 ...   
898           8.701813                   7   
916           8.010300                   7   
918           8.494850                   7   
935           4.978176                   7   
936           4.956130                   7   

                      properties.headline  
3     5 solar plant(s). Capacity: 94.9 MW  
5     4 solar plant(s). Capacity: 63.1 MW  
11   3 solar plant(s). Capacity: 270.0 MW  
14    3 solar plant(s). Capacity: 20.1 MW  
20    3 solar plant(s). Capacity: 35.0 MW  
..                                    ...  
898   1 solar plant(s). Capacity: 55.0 MW  
916   1 solar plant(s). Capacity: 40.0 MW  
918   1 solar plant(s). Capacity: 50.0 MW  
935    1 solar plant(s). Capacity: 9.9 MW  
936    1 solar plant(s). Capacity: 9.8 MW  

[178 rows x 8 columns]

Great! Now we have built a out all of our special properties for the template.

It's time to turn this back into **json format** we orient by records because that gives us an array of dictionaries.


In [29]:
ok_json = json.loads(df.to_json(orient='records'))


In [30]:
ok_json

[{'properties.COUNT': 6,
  'geometry.coordinates': '[129.0464035, 37.0845505]',
  'capacity_MW': 121.2,
  'properties.group_name': 'gas',
  'properties.article': "<div class='plant_list'><P>Type: gas\nCapacity: 19.8 MW\nAddress: 강원도  태백시  동점동  동점산업단지  블록  3-4(태백시  동점산업단지내)\nCompany: 티비에너지\nOwner: 김동현\nApproved year: 2021</p><p> Type: gas\nCapacity: 21.0 MW\nAddress: 강원도  태백시  동점동  동점산업단지  블록  3-5(태백시  동점산업단지내)\nCompany: 동도에너지\nOwner: 이사영\nApproved year: 2022</p><p> Type: gas\nCapacity: 19.8 MW\nAddress: 강원도  태백시  동점동  동점산업단지  블록  3-6(동점산업단지내)\nCompany: 동점그린에너지\nOwner: 김선웅\nApproved year: 2021</p><p> Type: gas\nCapacity: 19.8 MW\nAddress: 강원도  태백시  동점동  동점산업단지  블록  3-6(동점산업단지내)\nCompany: 동점그린에너지\nOwner: 정려안\nApproved year: 2021</p><p> Type: gas\nCapacity: 19.8 MW\nAddress: 강원도 태백시 동점동 동점산업단지 블록 3-4(태백시 동점산업단지내)\nCompany: 티비에너지\nOwner: 김선웅\nApproved year: 2021</p><p> Type: gas\nCapacity: 21.0 MW\nAddress: 강원도태백시동점동동점산업단지블록3-5(태백시동점산업단지내)\nCompany: 가진\nOwner: 정용주\nApproved year: 2022</P><

But because we had to normalize the hierarchy of the geojson document we now have to rebuild the hierarchy so this json document becomes geojson, the function below does just that:


In [31]:
def process_to_geojson(file):
    geo_data = {"type": "FeatureCollection", "features":[]}
    for row in file:
        this_dict = {"type": "Feature", "properties":{}, "geometry": {"type": "Point"}}
        for key, value in row.items():
            key_names = key.split('.')
            if key_names[0] == 'geometry':
                temp = value.replace('[', '').replace(']', '')
                list_values = temp.split(",")
                this_dict['geometry'][key_names[1]] = [float(val) for val in list_values]
            if str(key_names[0]) == 'properties':
                this_dict['properties'][key_names[1]] = value
        geo_data['features'].append(this_dict)
    return geo_data


In [32]:
geo_format = process_to_geojson(ok_json)

Now we can export this to a file! And don't forget to open it and prepend this variable assignment to the beginning of the file:

`infoData = `

In [33]:
#Variable name
with open('geo-data12-11-12am.js', 'w') as outfile:
    outfile.write("var infoData = ")
#geojson output
with open('geo-data12-11-12am.js', 'a') as outfile:
    json.dump(geo_format, outfile)


### Summary of Steps for mapping

1. Reduce your data frames to geographic units and groups
2. Create your properties: properties.name, properties.headline
3. properties.article: This is the most challenging one. You can potentially have the entire full-length articles here or many grouped/aggregated rows.
4. color, size, and rating properties: These are ways to visually represent quantities. You want to create a color palette or a range of colors and or sizes to represent quantities in your data frame.
5. Obtain geometries: See the Google Maps API notebook for individual latitude and longitude, or obtain shape files.
6. Merge geometries to your final output (this will likely be time consuming), and export to geojson.
7. Put the updated geojson file in the map template folder (replacing the old one). It must be named geo_data.js for the map to read it.
8. Customize Mapbox (see above and tutorial)


